# Exercice 1: Requêtes simples sur les données de Yago

In [ ]:
case class Triple(sujet: String, prop: String, objet: String)

val yagoFile = "/tmp/BDLE/dataset/yagoFacts5M.txt"




defined class Triple
yagoFile: String = /tmp/BDLE/dataset/yagoFacts5M.txt


In [ ]:
val yago = sc.textFile(yagoFile).
  map(ligne => ligne.split("\\t")).coalesce(8).
  map(tab => Triple(tab(0), tab(1), tab(2))).toDS()

yago.persist
yago.count


yago: org.apache.spark.sql.Dataset[Triple] = [sujet: string, prop: string ... 1 more field]
res4: Long = 5081764


5081764

## R1: Les régions dont la capitale (régionale) est Nantes

 `?x <hasCapital> <Nantes> `

La requête s'écrit ainsi avec une opération de sélection where, un renommage withColumnRenamed et une projection select :


In [ ]:

val r1 = yago.where("prop = '<hasCapital>' and objet = '<Nantes>'").
    withColumnRenamed("sujet","x").
    select("x")
r1.show(5)

+-------------------+
|                  x|
+-------------------+
|<Duchy_of_Brittany>|
| <Pays_de_la_Loire>|
+-------------------+

r1: org.apache.spark.sql.DataFrame = [x: string]


## R2: La fiche de Barack Obama

Afficher toutes les paires (propriété,objet) au sujet de `<Barack_Obama>`

 Le résultat doit être :
 
``
|        p       |         o          |
+----------------+--------------------+
| <graduatedFrom>|<Columbia_University|
| <graduatedFrom>|<Harvard_Law_School>|
|     <hasGender>|              <male>|
|   <hasWonPrize>|      <Grammy_Award>|
|   <hasWonPrize>| <Nobel_Peace_Prize>|
|<isAffiliatedTo>|<Democratic_Party>  |
``


In [ ]:
val r2 = yago.where("sujet='<Barack_Obama>'").
     withColumnRenamed("prop","p").
     withColumnRenamed("objet", "o").
     select("p", "o")

r2.show(5)

+---------------+--------------------+
|              p|                   o|
+---------------+--------------------+
|<graduatedFrom>|<Columbia_Univers...|
|<graduatedFrom>|<Harvard_Law_School>|
|    <hasGender>|              <male>|
|  <hasWonPrize>|      <Grammy_Award>|
|  <hasWonPrize>| <Nobel_Peace_Prize>|
+---------------+--------------------+
only showing top 5 rows

r2: org.apache.spark.sql.DataFrame = [p: string, o: string]


## R3: Les leaders parisiens

La requête est une étoile formée de deux triplets

``
    ?x <livesIn> <Paris> .
    ?x <isLeaderOf> ?z
``

Le résultat doit être:

``
|         x          |          z         |
+--------------------+--------------------+
|       <Louis_Blanc>|<Republican_Union...|
|<Stefano_Zacchiroli>| <Software_Heritage>|
+--------------------+--------------------+
``



In [ ]:
val livesInParis = yago.where("prop='<livesIn>' and objet='<Paris>'").
    withColumnRenamed("sujet", "x").
    select("x")

//livesInParis.show(5)

val leaders = yago.where("prop='<isLeaderOf>'").
    withColumnRenamed("sujet", "x").
    withColumnRenamed("objet", "z").
    select("x", "z")

//leaders.show(5)

val leadersInParis = livesInParis.join(leaders, "x")

leadersInParis.show(5)

+--------------------+--------------------+
|                   x|                   z|
+--------------------+--------------------+
|       <Louis_Blanc>|<Republican_Union...|
|<Stefano_Zacchiroli>| <Software_Heritage>|
+--------------------+--------------------+

livesInParis: org.apache.spark.sql.DataFrame = [x: string]
leaders: org.apache.spark.sql.DataFrame = [x: string, z: string]
leadersInParis: org.apache.spark.sql.DataFrame = [x: string, z: string]


## R4: Les joueurs du royaume uni.

La requête est un chemin formé de deux triplets

``
?x <playsFor> ?y .
?y <isLocatedIn> <United_Kingdom>
``

Les 5 premiers éléments du résultat sont :

``
|         y          |       x            |
+--------------------+--------------------+
|   <Royal_Air_Force>|      <George_Ayres>|
|   <Royal_Air_Force>|<Jack_Jones_(foot...|
|   <Royal_Air_Force>|<John_Hinton_(foo...|
|<University_of_Ed...|<James_Craigen_(f...|
|<University_of_Ed...|    <Ronald_Brebner>|
+--------------------+--------------------+
``


In [ ]:
val playsFor = yago.where("prop='<playsFor>'").
    withColumnRenamed("sujet", "x").
    withColumnRenamed("objet", "y").
    select("y", "x")

val inUK = yago.where("prop='<isLocatedIn>' and objet='<United_Kingdom>'").
    withColumnRenamed("sujet", "y").
    select("y")

val playersInUK = playsFor.join(inUK, "y")

playersInUK.show(5)

+--------------------+--------------------+
|                   y|                   x|
+--------------------+--------------------+
|   <Royal_Air_Force>|      <George_Ayres>|
|   <Royal_Air_Force>|<Jack_Jones_(foot...|
|   <Royal_Air_Force>|<John_Hinton_(foo...|
|<University_of_Ed...|<James_Craigen_(f...|
|<University_of_Ed...|    <Ronald_Brebner>|
+--------------------+--------------------+
only showing top 5 rows

playsFor: org.apache.spark.sql.DataFrame = [y: string, x: string]
inUK: org.apache.spark.sql.DataFrame = [y: string]
playersInUK: org.apache.spark.sql.DataFrame = [y: string, x: string]



## R5: Les acteurs qui influencent des créateurs

La requête est un flocon (ou snowflake) formé de 5 triplets :

``
   ?x <isCitizenOf> ?y .
   ?x <actedIn> ?z .
   ?x <influences> ?t .
   ?t <livesIn> ?u .
   ?t <created> ?v
``

Le résultat doit être:

``
+--------------+----------+---------+-----------------+--------------------+----------------+
|             t|         x|        y|                z|                   u|               v|
+--------------+----------+---------+-----------------+--------------------+----------------+
|<Irvine_Welsh>|<Iggy_Pop>|<Germany>|       <Cry-Baby>|            <Dublin>|<Dockers_(film)>|
|<Irvine_Welsh>|<Iggy_Pop>|<Germany>|       <Cry-Baby>|<Republic_of_Irel...|<Dockers_(film)>|
|<Irvine_Welsh>|<Iggy_Pop>|<Germany>|    <Suck_(film)>|            <Dublin>|<Dockers_(film)>|
|<Irvine_Welsh>|<Iggy_Pop>|<Germany>|    <Suck_(film)>|<Republic_of_Irel...|<Dockers_(film)>|
|<Irvine_Welsh>|<Iggy_Pop>|<Germany>|<Hardware_(film)>|            <Dublin>|<Dockers_(film)>|
|<Irvine_Welsh>|<Iggy_Pop>|<Germany>|<Hardware_(film)>|<Republic_of_Irel...|<Dockers_(film)>|
|<Irvine_Welsh>|<Iggy_Pop>|<Germany>|       <Dead_Man>|            <Dublin>|<Dockers_(film)>|
|<Irvine_Welsh>|<Iggy_Pop>|<Germany>|       <Dead_Man>|<Republic_of_Irel...|<Dockers_(film)>|
|<Irvine_Welsh>|<Iggy_Pop>|<Germany>|   <Gimme_Danger>|            <Dublin>|<Dockers_(film)>|
|<Irvine_Welsh>|<Iggy_Pop>|<Germany>|   <Gimme_Danger>|<Republic_of_Irel...|<Dockers_(film)>|
+--------------+----------+---------+-----------------+--------------------+----------------+
``

In [ ]:
val citizens = yago.where("prop='<isCitizenOf>'").
    withColumnRenamed("sujet", "x").
    withColumnRenamed("objet", "y").
    select("x", "y")

val actors = yago.where("prop='<actedIn>'").
    withColumnRenamed("sujet", "x").
    withColumnRenamed("objet", "z").
    select("x", "z")

val influences = yago.where("prop='<influences>'").
    withColumnRenamed("sujet", "x").
    withColumnRenamed("objet", "t").
    select("x", "t")

val livesIn = yago.where("prop='<livesIn>'").
    withColumnRenamed("sujet", "t").
    withColumnRenamed("objet", "u").
    select("t", "u")

val created = yago.where("prop='<created>'").
    withColumnRenamed("sujet", "t").
    withColumnRenamed("objet", "v").
    select("t", "v")


val r6 = citizens.join(actors, "x").
    join(influences, "x").
    join(livesIn, "t").
    join(created, "t")

r6.show(10)


+--------------+----------+---------+-----------------+--------------------+----------------+
|             t|         x|        y|                z|                   u|               v|
+--------------+----------+---------+-----------------+--------------------+----------------+
|<Irvine_Welsh>|<Iggy_Pop>|<Germany>|       <Cry-Baby>|            <Dublin>|<Dockers_(film)>|
|<Irvine_Welsh>|<Iggy_Pop>|<Germany>|       <Cry-Baby>|<Republic_of_Irel...|<Dockers_(film)>|
|<Irvine_Welsh>|<Iggy_Pop>|<Germany>|    <Suck_(film)>|            <Dublin>|<Dockers_(film)>|
|<Irvine_Welsh>|<Iggy_Pop>|<Germany>|    <Suck_(film)>|<Republic_of_Irel...|<Dockers_(film)>|
|<Irvine_Welsh>|<Iggy_Pop>|<Germany>|<Hardware_(film)>|            <Dublin>|<Dockers_(film)>|
|<Irvine_Welsh>|<Iggy_Pop>|<Germany>|<Hardware_(film)>|<Republic_of_Irel...|<Dockers_(film)>|
|<Irvine_Welsh>|<Iggy_Pop>|<Germany>|       <Dead_Man>|            <Dublin>|<Dockers_(film)>|
|<Irvine_Welsh>|<Iggy_Pop>|<Germany>|       <Dead_Man>|<Repu

## R6: Les paires de personnes mariées et nées dans le même pays

``
?x <isMarriedTo> ?y
?x <wasBornIn> ?n
?y <wasBornIn> ?n
``

Problème de la relation `?x <wasBornIn> ?n` avec `?n` qui peut être une ville, un pays...

In [ ]:
val marriages = yago.where("prop='<isMarriedTo>'").
    withColumnRenamed("sujet", "x").
    withColumnRenamed("objet", "y").
    select("x", "y")

val bornCity = yago.where("prop='<wasBornIn>'").
    withColumnRenamed("sujet", "x").
    withColumnRenamed("objet", "c").
    select("x", "c")

val nation = yago.where("prop='<hasCapital>'").
    withColumnRenamed("sujet", "n").
    select("n")

val locations = yago.where("prop='<isLocatedIn>'").
    withColumnRenamed("sujet", "c").
    withColumnRenamed("objet", "n").
    select("c", "n")

val cities = locations.join(nation, "n")
cities.show(5)

val bornNation = bornCity.join(cities, "c").
    select("x", "n")
bornNation.show(5)

/* val bornIn = yago.where("prop='<isCitizenOf>'").
    withColumnRenamed("sujet", "x").
    withColumnRenamed("objet", "n").
    select("x", "n")
*/

val r6 = marriages.join(bornNation, "x").
    join(bornNation.toDF("y", "n2"), "y").
    filter($"n" === $"n2").
    select("y", "x", "n")

r6.show(10)
    
//TODO: Select only countries in bornNation

+--------+------------------+
|       n|                 c|
+--------+------------------+
|<Aargau>|            <Aare>|
|<Aargau>|<Bruno_Weber_Park>|
|<Aargau>|    <Lake_Hallwil>|
|<Aargau>|          <Limmat>|
|<Aargau>|          <Lägern>|
+--------+------------------+
only showing top 5 rows

+--------------------+-----------------+
|                   x|                n|
+--------------------+-----------------+
|    <Edwin_Retamoso>|           <Peru>|
|    <Edwin_Retamoso>|<Apurímac_Region>|
|    <Edwin_Retamoso>|<Apurímac_Region>|
|<James_Connolly_(...|     <Queensland>|
|<James_Connolly_(...|      <Australia>|
+--------------------+-----------------+
only showing top 5 rows

+--------------------+--------------------+---------------+
|                   y|                   x|              n|
+--------------------+--------------------+---------------+
|   <Caterina_Sforza>|<Giovanni_il_Popo...|        <Italy>|
|<Cornelius_Vander...|<Marie_Norton_Har...|     <New_York>|
|          

## R7: Les personnes influencées par un prix nobel

## R8: L'acteur ayant joué dans le plus grand nombre de films